# **Reduction of Quantum Circuits using CLUE**$\def\bx{\mathbf{x}}\def\quant#1{\left|#1\right\rangle}\def\cc{\mathbb{C}}$

In [9]:
import sys; sys.path.insert(0, "../..") # clue is here
from clue.qiskit import *
from numpy import matmul, cdouble

## 1. Description of methodology

In this notebook we show how we can use CLUE to reduce a Quantum circuit described in `qasm`. Without really understanding what is going on under the hood, we know that, given a quantum circuit there is a unitary matrix associated that represents the quantum computations that are done in the circuit.

In my current intuition, if the circuit hsa $n$ q-bits, then there is a complex $2^n\times 2^n$ matrix $A$ that act like a "Markov matrix" over a valid superposition of states. Namely, if we have the state $\bx = (x_0,\ldots,x_{2^n-1})$ where the $x_i$ entry represent the part of the superposition corresponding with the state $\quant{i}$, then we obtain the state $A\bx^T$ after applying the quantum circuit.

We would like to get a reduction (lumping) $L \in \cc^{m\times 2^n}$ for this matrix $A$ that helps to reduce the size or simplify the understanding of the circuit. For doing so we do the following:

1. Read a `qasm` file as taken from the benchmarks in this website: [Benchmarks](https://www.cda.cit.tum.de/mqtbench/)
2. We compute the unitary matrix $A$ for the circuit:
   * We need to remove all measures from the `qasm` file.
   * It seems the software `qiskit` does simulations over the circuit in order to compute this unitary matrix.
3. We use this matrix to set up a dynamical system of the form $\bx_{k+1}^T = A\bx_k^T$ (this system is not actually happening, but may be interesting to remember).
4. We run CLUE for this system for a given observable (currently we take one of the variables) $\longrightarrow$ we get a matrix $L$.
   * We run *approximate CLUE* since the unitary matrix has double precision complex numbers as inputs.
   * We have adapted the code of CLUE to *run over the complex numbers* too.
   * In some cases, when we get a reduction it is an exact lumping (at least in the QFT models that has been checked by hand).
   * The matrix $L$ is orthonormal, meaning that the reduced system is defined again by a unitary matrix.

## 2. Some examples 

Let us show in the code how to compute these reductions using CLUE for the QFT algorithm with 3 q-bits. 

**Remark**: the `qasm` files are included in the folder `circuits`.

In [5]:
system = DS_QuantumCircuit("circuits/qft_indep_qiskit_3.qasm"); system

circuit-119 [DS_QuantumCircuit -- 8 -- SparsePolynomial]

### The data in the system

We can show the equations of the system:

In [7]:
for (i,equ) in enumerate(system.equations): 
    print(i, " -> ", equ)

0  ->  (0.353553390593274 + 0.0j)*Q_000 + (0.353553390593274 - 4.32978028117747e-17j)*Q_001 + (0.353553390593274 - 4.32978028117747e-17j)*Q_010 + (0.353553390593274 - 8.65956056235493e-17j)*Q_011 + (0.353553390593274 - 4.32978028117747e-17j)*Q_100 + (0.353553390593274 - 8.65956056235493e-17j)*Q_101 + (0.353553390593274 - 8.65956056235493e-17j)*Q_110 + (0.353553390593274 - 1.29893408435324e-16j)*Q_111
1  ->  (0.353553390593274 + 0.0j)*Q_000 + (0.25 + 0.25j)*Q_001 + (6.4946704217662e-17 + 0.353553390593274j)*Q_010 + (-0.25 + 0.25j)*Q_011 + (-0.353553390593274 + 4.32978028117747e-17j)*Q_100 + (-0.25 - 0.25j)*Q_101 + (-1.08244507029437e-16 - 0.353553390593274j)*Q_110 + (0.25 - 0.25j)*Q_111
2  ->  (0.353553390593274 + 0.0j)*Q_000 + (6.4946704217662e-17 + 0.353553390593274j)*Q_001 + (-0.353553390593274 + 4.32978028117747e-17j)*Q_010 + (-1.08244507029437e-16 - 0.353553390593274j)*Q_011 + (0.353553390593274 - 4.32978028117747e-17j)*Q_100 + (1.08244507029437e-16 + 0.353553390593274j)*Q_101 + (-

Or, since the system is linear, we can show the unitary matrix from the system:

In [13]:
A = system.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble)
for (i,row) in enumerate(A.round(5)):
    print(i, " -> ", row)

0  ->  [0.35355+0.j 0.35355+0.j 0.35355+0.j 0.35355+0.j 0.35355+0.j 0.35355+0.j
 0.35355+0.j 0.35355+0.j]
1  ->  [ 0.35355-0.j       0.25   +0.25j     0.     +0.35355j -0.25   +0.25j
 -0.35355+0.j      -0.25   -0.25j    -0.     -0.35355j  0.25   -0.25j   ]
2  ->  [ 0.35355-0.j       0.     +0.35355j -0.35355+0.j      -0.     -0.35355j
  0.35355-0.j       0.     +0.35355j -0.35355+0.j      -0.     -0.35355j]
3  ->  [ 0.35355-0.j      -0.25   +0.25j    -0.     -0.35355j  0.25   +0.25j
 -0.35355+0.j       0.25   -0.25j     0.     +0.35355j -0.25   -0.25j   ]
4  ->  [ 0.35355-0.j -0.35355+0.j  0.35355-0.j -0.35355+0.j  0.35355-0.j
 -0.35355+0.j  0.35355-0.j -0.35355+0.j]
5  ->  [ 0.35355-0.j      -0.25   -0.25j     0.     +0.35355j  0.25   -0.25j
 -0.35355+0.j       0.25   +0.25j    -0.     -0.35355j -0.25   +0.25j   ]
6  ->  [ 0.35355-0.j      -0.     -0.35355j -0.35355+0.j       0.     +0.35355j
  0.35355-0.j      -0.     -0.35355j -0.35355+0.j       0.     +0.35355j]
7  ->  [ 0.35355-0.

### Computing the lumping

Let say we want to compute the lumping with respect to the state $\quant{100} = \quant{4}$. We can do that with the following code:

In [18]:
lumped = system.lumping([system.variables[4]])

New variables:
y0 = Q_100
y1 = (0.377964473009227 + 0.0j)*Q_000 + (-0.377964473009227 - 4.62872982062167e-17j)*Q_001 + (0.377964473009227 + 4.62872982062166e-17j)*Q_010 + (-0.377964473009227 - 9.25745964124333e-17j)*Q_011 + (-0.377964473009227 - 9.25745964124333e-17j)*Q_101 + (0.377964473009227 + 9.25745964124332e-17j)*Q_110 + (-0.377964473009227 - 1.3886189461865e-16j)*Q_111
New initial conditions:
Lumped system:
y0' = (0.935414346693485 + 7.8702812005206e-33j)*y1 + (0.353553390593274 - 4.32978028117747e-17j)*y0
y1' = (-0.353553390593274 - 7.32847151738175e-17j)*y1 + (0.935414346693485 + 6.54601248888394e-17j)*y0


We can get the lumping matrix (as read from the previous output):

In [20]:
L = lumped.lumping_matrix.to_numpy(dtype=cdouble); L

array([[ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         1.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.37796447+0.00000000e+00j, -0.37796447-4.62872982e-17j,
         0.37796447+4.62872982e-17j, -0.37796447-9.25745964e-17j,
         0.        +0.00000000e+00j, -0.37796447-9.25745964e-17j,
         0.37796447+9.25745964e-17j, -0.37796447-1.38861895e-16j]])

And we can check that the matrix for the new system is unitary:

In [23]:
A = lumped.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble)
matmul(A, A.transpose().conjugate()).round(15)

array([[1.+0.j, 0.+0.j],
       [0.-0.j, 1.+0.j]])

## 3. More experiments?

This place is for further experimentation on the fly